In [1]:
from acdc.transprop.utils import get_finetuned_gpt2

model = get_finetuned_gpt2()

Loaded pretrained model sachalmalick/gpt2-transprop-ft-welterweight into HookedTransformer
Moving model to device:  cuda


In [2]:
import acdc.transprop.utils as tp
import importlib
importlib.reload(tp)
transprop_things = tp.get_transprop_things(20, "frac_correct")
data = tp.get_prompt_data(5, model)

Loaded pretrained model sachalmalick/gpt2-transprop-ft-welterweight into HookedTransformer
Moving model to device:  cuda
torch.Size([10, 50257])


In [3]:
import torch

ds = transprop_things.validation_data
model = transprop_things.tl_model
output = transprop_things.tl_model(ds)
indices = torch.sum(transprop_things.validation_mask, dim=1)
logits = torch.stack([
    torch.squeeze(output[i, indices[i] - 1, :]) for i in range(indices.shape[0])
])
print(logits.shape)
top_indices = torch.topk(logits, k=5, dim=-1).indices[9]
print(top_indices)
predicted_words = [model.tokenizer.decode([idx.item()]) for idx in top_indices]
print(predicted_words)
print(model.tokenizer.decode(ds[9]))
print(transprop_things.validation_labels[9])
print(transprop_things.validation_metric(output))
print(transprop_things.validation_labels[9])
print(model.tokenizer.decode(transprop_things.validation_labels[9]))

model.eval()

from torch.nn import CrossEntropyLoss
loss_fn = CrossEntropyLoss()
loss = loss_fn(logits, transprop_things.validation_labels)
wrong_loss = loss_fn(logits, transprop_things.validation_labels)
print("Cross entropy loss", loss, wrong_loss)
# # Define a prompt
prompt = "If all apples are fruits and all fruits are edible, then it can be inferred that all apples are"

# # Tokenize the prompt
input_ids = model.tokenizer.encode(prompt, return_tensors="pt")

# # Generate the next word
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs[:, -1, :]  # Take the logits of the last position
    top_indices = torch.topk(logits, k=5, dim=-1).indices[0]


# # Decode the predicted index to get the next word
predicted_words = [model.tokenizer.decode([idx.item()]) for idx in top_indices]

print("Next word prediction:", predicted_words)

torch.Size([10, 50257])
tensor([31853,  5657,   540,   257, 20690], device='cuda:0')
[' barric', 'bar', 'able', ' a', ' siege']
banana implies comb and if comb then barricade therefore by the transitive property banana also implies<|endoftext|><|endoftext|>
tensor(31853, device='cuda:0')
torch.Size([10, 21, 50257])
torch.Size([10])
torch.Size([10])
tensor([29.1176, 18.2405, 25.2618, 12.1479, 31.6272, 15.0748, 25.9485, 12.0949,
        42.0337, 26.0337], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([ 3.4341, -6.2562,  1.2916, -4.8504, -2.1465,  0.6659, 10.6417, -6.2371,
         3.0601,  5.4880], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([True, True, True, True, True, True, True, True, True, True],
       device='cuda:0')
tensor(-1., device='cuda:0')
tensor(31853, device='cuda:0')
 barric
Cross entropy loss tensor(1.2521, device='cuda:0', grad_fn=<NllLossBackward0>) tensor(1.2521, device='cuda:0', grad_fn=<NllLossBackward0>)
Next word prediction: [' edible', 'ible', 'ed', '

In [4]:
import acdc.transprop.utils as tp
import importlib
importlib.reload(tp)
from acdc.ioi.utils import get_gpt2_small
import gc
def perform_baseline_evaluation(metrics, model, prompts):
    results = {}
    for prompt in prompts:
        metric_results = {}
        for metric in metrics:
            transprop_things = tp.get_transprop_things(20,
                                                    metric,    
                                                    model=model,
                                                    prompt=prompt)
            ds = transprop_things.validation_data
            output = transprop_things.tl_model(ds)
            metric_result = transprop_things.validation_metric(output)
            metric_results[metric] = metric_result
            model.reset_hooks()
        results[prompt] = metric_results
    return results

In [14]:
model = get_gpt2_small()

Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda


In [6]:
prompt_templates = [
    "{a} implies {b} and if {b} then {c} therefore by the transitive property {a} also implies",
    "if all {a} are {b} and {b} are {c} then all {a} are",
    "if all {a} are {b} and {b} is a type of {c} then it can be inferred that all {a} are a type of",
    "{a} implies {b} and if {b} then {c} therefore {a} also implies",
    "{a} implies {b} and if {b} then {c} therefore by the transitive property {a} also implies",
    "{a} implies {b} and {b} implies {c} then by the transitive property {a} also implies",
    "{a} is a type of {b} and all {b} are {c} therefore {a} is also a type of"
]
baseline = perform_baseline_evaluation(["kl_div"], model, prompt_templates)
print(baseline)

torch.Size([10, 50257])
torch.Size([10, 50257])
torch.Size([10, 50257])
torch.Size([10, 50257])


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 21.99 GiB total capacity; 21.13 GiB already allocated; 1024.00 KiB free; 21.64 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF